# Neural Network Prediction of Van der Pol Oscillator

**Team 19**: Vlad-Flavius Misăilă, Robert-Daniel Man, Sebastian-Adrian Mărginean

## Overview

The Van der Pol oscillator is a nonlinear oscillator with nonlinear damping that exhibits limit cycle behavior:

$$
\frac{d^2x}{dt^2} - \mu(1 - x^2)\frac{dx}{dt} + x = 0
$$

As a first-order system:
$$
\begin{align}
\frac{dx}{dt} &= y \\
\frac{dy}{dt} &= \mu(1 - x^2)y - x
\end{align}
$$

**Parameter**: $\mu$ controls the nonlinearity and strength of damping
- $\mu = 0$: Simple harmonic oscillator
- $\mu > 0$: Self-excited oscillations with limit cycle

**Objective**: Predict oscillatory dynamics and analyze limit cycle behavior.

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import matplotlib.pyplot as plt
import torch

from src.dynamical_systems import VanDerPolOscillator
from src.data_preparation import generate_trajectory, create_sequences
from src.neural_models import FeedForwardPredictor, LSTMPredictor, GRUPredictor, NeuralPredictor
from src.evaluation import (
    evaluate_prediction, plot_trajectory_2d, plot_time_series,
    plot_training_history, plot_phase_space
)

np.random.seed(42)
torch.manual_seed(42)
plt.rcParams['figure.dpi'] = 100
%matplotlib inline

print("✓ Setup complete!")

## 1. Generate Van der Pol Oscillator Data

In [ ]:
# Test different mu values
mu_values = [0.5, 1.0, 2.0, 5.0]
trajectories = {}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, mu in enumerate(mu_values):
    vdp = VanDerPolOscillator(mu=mu)
    t, traj = generate_trajectory(
        vdp,
        initial_state=np.array([2.0, 0.0]),
        t_span=(0, 50),
        dt=0.01
    )
    trajectories[mu] = (t, traj)
    
    # Plot phase portrait
    axes[idx].plot(traj[:, 0], traj[:, 1], 'b-', linewidth=1.5, alpha=0.7)
    axes[idx].set_xlabel('x (position)', fontsize=11)
    axes[idx].set_ylabel('y (velocity)', fontsize=11)
    axes[idx].set_title(f'Phase Portrait (μ = {mu})', fontsize=12)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].axhline(0, color='k', linewidth=0.5)
    axes[idx].axvline(0, color='k', linewidth=0.5)

plt.tight_layout()
plt.suptitle('Van der Pol Oscillator: Effect of μ Parameter', fontsize=14, y=1.02)
plt.show()

print("✓ Generated trajectories for different μ values")

## 2. Focus on μ = 2.0 (Moderate Nonlinearity)

In [ ]:
# Use mu = 2.0 for detailed analysis
vdp = VanDerPolOscillator(mu=2.0)
initial_state = np.array([2.0, 0.0])
t, trajectory = generate_trajectory(
    vdp,
    initial_state=initial_state,
    t_span=(0, 100),
    dt=0.01
)

print(f"✓ Generated {len(t)} time points")
print(f"✓ Trajectory shape: {trajectory.shape}")

In [ ]:
# Visualize time series and phase space
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Time series
axes[0].plot(t[:1000], trajectory[:1000, 0], 'b-', linewidth=1.5, label='Position (x)')
axes[0].plot(t[:1000], trajectory[:1000, 1], 'r-', linewidth=1.5, label='Velocity (y)')
axes[0].set_xlabel('Time', fontsize=12)
axes[0].set_ylabel('Value', fontsize=12)
axes[0].set_title('Van der Pol Time Series (μ = 2.0)', fontsize=13)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Phase portrait with limit cycle
axes[1].plot(trajectory[:, 0], trajectory[:, 1], 'b-', linewidth=1, alpha=0.6)
axes[1].plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=10, label='Start')
axes[1].plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=10, label='End')
axes[1].set_xlabel('x (position)', fontsize=12)
axes[1].set_ylabel('y (velocity)', fontsize=12)
axes[1].set_title('Phase Portrait: Convergence to Limit Cycle', fontsize=13)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(0, color='k', linewidth=0.5)
axes[1].axvline(0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

## 3. Prepare Data and Train Models

In [ ]:
# Prepare sequences
WINDOW_SIZE = 50
X_train, y_train, X_test, y_test, scaler = create_sequences(
    trajectory, window_size=WINDOW_SIZE, train_ratio=0.8, normalize=True
)

print(f"Training: X={X_train.shape}, y={y_train.shape}")
print(f"Testing: X={X_test.shape}, y={y_test.shape}")

In [ ]:
# Train Feed-Forward Network
print("\nTraining Feed-Forward Network...")
fnn = FeedForwardPredictor(WINDOW_SIZE * 2, [64, 32], 2, dropout=0.1)
fnn_predictor = NeuralPredictor(fnn, learning_rate=0.001)
fnn_history = fnn_predictor.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=32)

plot_training_history(fnn_history, 'FNN Training (Van der Pol)')

In [ ]:
# Train LSTM Network
print("\nTraining LSTM Network...")
lstm = LSTMPredictor(2, 32, 2, 2, dropout=0.1)
lstm_predictor = NeuralPredictor(lstm, learning_rate=0.001)
lstm_history = lstm_predictor.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=32)

plot_training_history(lstm_history, 'LSTM Training (Van der Pol)')

In [ ]:
# Train GRU Network (additional comparison)
print("\nTraining GRU Network...")
gru = GRUPredictor(2, 32, 2, 2, dropout=0.1)
gru_predictor = NeuralPredictor(gru, learning_rate=0.001)
gru_history = gru_predictor.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=32)

plot_training_history(gru_history, 'GRU Training (Van der Pol)')

## 4. Evaluate and Compare Models

In [ ]:
# Make predictions
fnn_pred = fnn_predictor.predict(X_test)
lstm_pred = lstm_predictor.predict(X_test)
gru_pred = gru_predictor.predict(X_test)

# Evaluate
fnn_metrics = evaluate_prediction(y_test, fnn_pred)
lstm_metrics = evaluate_prediction(y_test, lstm_pred)
gru_metrics = evaluate_prediction(y_test, gru_pred)

print("="*60)
print("ONE-STEP PREDICTION RESULTS (Van der Pol Oscillator)")
print("="*60)
print(f"\nFeed-Forward: RMSE={fnn_metrics['rmse']:.6f}, MAE={fnn_metrics['mae']:.6f}")
print(f"LSTM:         RMSE={lstm_metrics['rmse']:.6f}, MAE={lstm_metrics['mae']:.6f}")
print(f"GRU:          RMSE={gru_metrics['rmse']:.6f}, MAE={gru_metrics['mae']:.6f}")
print("="*60)

In [ ]:
# Visualize one-step predictions in phase space
y_test_orig = scaler.inverse_transform(y_test)
lstm_pred_orig = scaler.inverse_transform(lstm_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# True trajectory
axes[0].plot(y_test_orig[:, 0], y_test_orig[:, 1], 'b-', linewidth=2, alpha=0.7, label='True')
axes[0].set_xlabel('x (position)', fontsize=12)
axes[0].set_ylabel('y (velocity)', fontsize=12)
axes[0].set_title('True Test Trajectory', fontsize=13)
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Predictions
axes[1].plot(y_test_orig[:, 0], y_test_orig[:, 1], 'b-', linewidth=2, alpha=0.5, label='True')
axes[1].plot(lstm_pred_orig[:, 0], lstm_pred_orig[:, 1], 'r--', linewidth=2, alpha=0.7, label='LSTM Pred')
axes[1].set_xlabel('x (position)', fontsize=12)
axes[1].set_ylabel('y (velocity)', fontsize=12)
axes[1].set_title('LSTM One-Step Predictions', fontsize=13)
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Long-Term Prediction: Limit Cycle Preservation

In [ ]:
# Multi-step prediction to test if model preserves limit cycle
initial_window = X_test[0]
n_steps = 500  # Predict ~5 oscillation cycles

true_future = y_test[:n_steps]
fnn_future = fnn_predictor.iterative_predict(initial_window, n_steps)
lstm_future = lstm_predictor.iterative_predict(initial_window, n_steps)
gru_future = gru_predictor.iterative_predict(initial_window, n_steps)

# Inverse transform
true_orig = scaler.inverse_transform(true_future)
fnn_orig = scaler.inverse_transform(fnn_future)
lstm_orig = scaler.inverse_transform(lstm_future)
gru_orig = scaler.inverse_transform(gru_future)

print(f"✓ Generated {n_steps}-step predictions")

In [ ]:
# Compare phase portraits
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# True
axes[0, 0].plot(true_orig[:, 0], true_orig[:, 1], 'b-', linewidth=2, alpha=0.7)
axes[0, 0].set_title('True Trajectory', fontsize=13)
axes[0, 0].set_xlabel('x')
axes[0, 0].set_ylabel('y')
axes[0, 0].grid(True, alpha=0.3)

# FNN
axes[0, 1].plot(fnn_orig[:, 0], fnn_orig[:, 1], 'r-', linewidth=2, alpha=0.7)
axes[0, 1].set_title('FNN Prediction', fontsize=13)
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('y')
axes[0, 1].grid(True, alpha=0.3)

# LSTM
axes[1, 0].plot(lstm_orig[:, 0], lstm_orig[:, 1], 'g-', linewidth=2, alpha=0.7)
axes[1, 0].set_title('LSTM Prediction', fontsize=13)
axes[1, 0].set_xlabel('x')
axes[1, 0].set_ylabel('y')
axes[1, 0].grid(True, alpha=0.3)

# GRU
axes[1, 1].plot(gru_orig[:, 0], gru_orig[:, 1], 'm-', linewidth=2, alpha=0.7)
axes[1, 1].set_title('GRU Prediction', fontsize=13)
axes[1, 1].set_xlabel('x')
axes[1, 1].set_ylabel('y')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle(f'{n_steps}-Step Iterative Predictions: Limit Cycle Preservation', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Overlay comparison
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(true_orig[:, 0], true_orig[:, 1], 'b-', linewidth=2.5, alpha=0.6, label='True')
ax.plot(fnn_orig[:, 0], fnn_orig[:, 1], 'r--', linewidth=1.5, alpha=0.7, label='FNN')
ax.plot(lstm_orig[:, 0], lstm_orig[:, 1], 'g--', linewidth=1.5, alpha=0.7, label='LSTM')
ax.plot(gru_orig[:, 0], gru_orig[:, 1], 'm--', linewidth=1.5, alpha=0.7, label='GRU')
ax.set_xlabel('x (position)', fontsize=12)
ax.set_ylabel('y (velocity)', fontsize=12)
ax.set_title(f'Comparison: {n_steps}-Step Prediction (Van der Pol)', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Summary

### Key Observations:

1. **Limit Cycle Behavior**: The Van der Pol oscillator exhibits a stable limit cycle
2. **Model Performance**: 
   - All models achieve good one-step prediction accuracy
   - Recurrent models (LSTM/GRU) better preserve limit cycle structure in long-term predictions
3. **Periodicity**: Unlike chaotic systems, oscillatory systems are more predictable
4. **Architecture Comparison**: LSTM and GRU perform similarly, both outperforming FNN

### Insights:

- **Oscillatory vs Chaotic**: Limit cycle systems are fundamentally more predictable than chaotic systems
- **Structural Preservation**: Neural networks can learn to preserve important dynamical features (limit cycles)
- **Parameter Effects**: The μ parameter controls relaxation oscillation strength

### Applications:

- Biological rhythms (heartbeat, neural oscillations)
- Electronic oscillator circuits
- Climate oscillations
- Chemical reaction kinetics

In [ ]:
print("\n✓ Van der Pol oscillator analysis complete!")